In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
def extract_run_time(trial):
    for line in trial.split('\n'):
        if 'Test took' in line:
            time_str = re.findall("\d+\.\d+", line)[0]
            time = float(time_str)
            # compute units
            unit = line[10 + len(time_str):]
            if unit == 'ms':
                time *= 1000
            if unit == 'us':
                time *= 1_000_000
            return time
    return 0.0


def extract_experiments(file_name):
    result = {}
    with open(file_name) as f:
        parsed_json = json.load(f)
        for expt in parsed_json.keys():
            trials = parsed_json[expt]
            trial_times = []
            for trial in trials:
                trial_times.append(extract_run_time(trial))
            avg = np.average(np.array(trial_times))
            result[expt] = avg

    return result


def aggregate_experiment(paxos_expts, mencius_expts, epaxos_expts):
    aggregate_expts = {}
    # Assumes each expriment shares the same key
    for expt in paxos_expts.keys():
        paxos_avg = paxos_expts[expt]
        mencius_avg = mencius_expts[expt]
        epaxos_avg = epaxos_expts[expt]

        aggregate_expts[expt] = ([paxos_avg, mencius_avg, epaxos_avg], ["paxos", "mencius", "epaxos"])
    
    return aggregate_expts



In [ ]:
def plot_expt(expt, aggregate_expts):
    (expt_avgs, expt_labels) = aggregate_expts[expt]

    # Define a list of colors for the bars
    colors = ['blue', 'green', 'red']

    # Create the bar plot
    plt.figure(figsize=(8, 6))  # Adjust figure size as desired
    bars = plt.bar(expt_labels, expt_avgs, color=colors)

    # Add labels and title
    plt.xlabel("Protocols")
    plt.ylabel("100,000 packet Completion time")
    plt.title("Skewed Workload Evaluation")

    # Set x-axis tick positions to center bars
    plt.xticks(range(len(expt_labels)), expt_labels, rotation=0)  # Adjust rotation for long labels

    # Display the graph
    plt.show()

In [ ]:
# Example plot

# Extract experiment data for all protocols
paxos_expts = extract_experiments("skewed_workload_paxos.json")
mencius_expts = extract_experiments("skewed_workload_mencius.json")
epaxos_expts = extract_experiments("skewed_workload_epaxos.json")

# aggregate different expts into format for plotting
aggregate_expts = aggregate_experiment(paxos_expts, mencius_expts, epaxos_expts)

# plot specific experiment of interest
plot_expt("-1,0", aggregate_expts)